# PTT 看板(Board) 

### Culture, Consumption, Politics, Gossip.....

In [ ]:
'''
Skills：
1.Web Crawler
2.DataPipeLine
3.Dockefile/Docker/DockerHub(port)
4.DataMining(numeric, text mining, image, opinion)
5.Machine Learning/Sentiment analysis/Deep learning
6.Text categorization: Topic minin, Sentiment anaylysis, Opinion mining
'''

# Deploy MongoDB docker

In [ ]:
#把爬取的資料部署裝到容器中, 也可在Linux中直接執行

In [8]:
#!curl -fsSL get.docker.com -o get-docker.sh | sh  #改以Linux部署docker容器

In [9]:
#!docker run -p 27017:27017 -v $(pwd)/mongodb_data:/data/db --name some-mongo -d mongo  #利用port號run一台mongodb的容器,把本地端資料餵進去

In [3]:
!pip install pymongo  #載入pymongo連線套件

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


#  Data Pipeline Design

In [5]:
#功能 (url位址即檔案)
'''
1.將request的網址自動寫入檔案
2.任意設定PTT看板(Board)分類與爬取筆數(Num),每個板約3000多個網頁/每頁約10-20筆文章
3.自動化擷取各檔案的頁面資訊,餵到雲端容器中
4.創BOW(Bag of Words)詞向量,依字頻排序（ex：英文部分）
.....
.....
'''

import requests
import re
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from collections import Counter  



def get_ptt(Board, Num):  
    main_page = 'https://www.ptt.cc/bbs/%s'%Board + '/index.html'  
    response = requests.get(main_page)
    total_page = int(response.text.split('a class="btn wide" href="/bbs/%s'%Board +'/index')[2].split('.html">&lsaquo; 上頁</a>')[0]) + 1
    url_template = 'https://www.ptt.cc/bbs/%s'%Board + '/index%s.html'    
    pages_to_crawl = Num  
    for page in range(total_page, total_page - pages_to_crawl,-1):  
        urls = url_template%page  
        return urls
    
    
def get_ptt_detail(urls, Board, Num):   
    with open('./ptt_data_all/%s'%Num, 'w') as f:   
        html=f.write(requests.get(urls, headers={"cookie":"over18=1"}).text)
    with open('./ptt_data_all/%s'%Num) as f: 
        html=f.read()
        detail_urls_list = re.findall('/bbs/%s'%Board + '/M.{18}html', html)
        return detail_urls_list  


def parse_ptt_detail(detail_urls_list):
    for i in range(len(detail_urls_list)):
        with open('./ptt_data_all/details/%s'%i, 'w') as f:   #linux可直接touch files
            html = f.write(requests.get('https://www.ptt.cc/'+ detail_urls_list[i]).text)
        with open('./ptt_data_all/details/%s'%i) as f:
            html = f.read()
            soup = BeautifulSoup(html, 'lxml') 
            metas = [tag.text for tag in soup.find_all('span', {"class":"article-meta-value"})] 
            ptt_data = {}
            ptt_data['author']  = metas[0]
            ptt_data['board'] = metas[1]
            ptt_data['title'] = metas[2]
            ptt_data['dt'] = metas[3]
            content = [tag.text for tag in soup.find_all('div', {"id":"main-content"}, {"class":"bbs-screen bbs-content"})] 
            ptt_data['content'] = content
            return ptt_data

    
#def save_to_mongo(ptt_data):   #塞入容器再壓成image, 或直接寫成docker file(sh)
#    client = MongoClient()
#    client.crawler.ptt.insert(ptt_data)  #crawler: database, ptt: collection
#    client = MongoClient()
#    text = client.crawler.ptt.get('content')
#    return client, text



#def cnt_words(text, res={}):  #進入容器, 取出文章後, 開始創BOW,或用R-progam斷詞套件(合併寫在docker file中)
#    bag = text.replace(',','').replace('.','').replace('?','').lower().split()
#    for word in bag:
#        if word not in stopwords:  #stopwords是另一篇文章
#            if word in res:
#                res[word] += 1    #統計wordf freq
#            else:
#                res[word] = 1
#    return res


In [ ]:
#假設設定要爬NBA板20頁

urls = get_ptt('NBA', 20)
detail_urls = get_ptt_detail(urls, 'NBA', 20)
ptt_data = parse_ptt_detail(detail_urls)
#save_to_mongo(ptt_data)
#res = cnt_words(speech, Counter())   
#res.most_common()   #word freq排序